# Fill necessary info

In [114]:
contractAddress = '0x0f4507E6dc60EADc0e9b0034Fd370A0f5F7B3B34'
name = "Vlados"
addr = '0x0bF0938fA40eaAA77B803467C2CEf44b49B04365'

### Connect to test net

In [115]:
from web3 import HTTPProvider, Web3
import web3
import json

web3 = Web3(HTTPProvider('http://127.0.0.1:7545'))
print("is connected:", web3.isConnected())

is connected: True


In [116]:
address = web3.toChecksumAddress(contractAddress)
web3.eth.defaultAccount = addr

In [117]:
trufflePath = 'contract'
truffleFile = json.load(open(trufflePath + '/build/contracts/Monopoly.json'))

abi = truffleFile['abi']

In [118]:
game = web3.eth.contract(address=address, abi=abi)

### Some functions

In [119]:
def isInPrison(contract, playerNumber):
    return contract.functions.getPlayerByIndex(playerNumber).call()[4] > 0

def getPlayers(contract):
    players = {}
    maxNumber = contract.functions.getMaxNumberOfPlayers().call()
    for i in range(maxNumber):
        player = contract.functions.getPlayerByIndex(i).call()
        players[player[1]]={"money":player[2], "position":player[3], "moves in prison":player[4]}
    return players

def getPositions(contract):
    positions = []
    numberOfPositions = contract.functions.getNumberOfPositions().call()
    players = getPlayers(contract)
    
    for i in range(numberOfPositions):
        [state, owner] = contract.functions.getPositionInfoByIndex(i).call()
        if state == 0:
            positions.append(["None",""])
        if state == 1:
            positions.append(["Available", ""])
        if state == 2:
            positions.append(["Bought", game.functions.getPlayerByIndex(owner).call()[1]])
    
    return positions

def printLogInfo(event_filter):
    events=event_filter.get_new_entries()
    for event in events:
        print("Action:",event['args']['name'],"at position",event['args']['position'],":", event['args']['action'])
        
def isInPrison(contract, playerNumber):
    return contract.functions.getPlayerByIndex(playerNumber).call()[4] > 0

### Enroll the game

In [120]:
print(game.functions.enroll(name).call())
tx_hash=game.functions.enroll(name).transact()
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
print("status",tx_receipt.status)

filterMoveIsMade = game.events.moveIsMade.createFilter(fromBlock="latest", argument_filters={'name':name})
filterStationBought = game.events.stationBought.createFilter(fromBlock="latest", argument_filters={'name':name})
filterActionHappened = game.events.actionHappened.createFilter(fromBlock="latest")

[isOk, playerNumber] = game.functions.getNumberOfYourPlayer().call()
if isOk:
    print("player number", playerNumber)
else:
    print("get number of player error")

True
status 1
player number 0


### Make move

In [47]:
print("game is active:", game.functions.isGameActive().call())
if game.functions.getPlayerByIndex(playerNumber).call()[5]:
    print("Need decision")
else:
    print("Don't need decision")
print("whose move:", game.functions.getWhoseMove().call())

game is active: True
Don't need decision
whose move: Vlados


In [48]:
if game.functions.getWhoseMove().call() == name:
    tx_hash=game.functions.makeMove().transact()
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    print("status",tx_receipt.status)
    events=filterMoveIsMade.get_new_entries()
    if len(events) > 0:
        print("Dice value:", events[len(events)-1]['args']['dice'])
    else:
        print("no events with moves")
    if game.functions.getPlayerByIndex(playerNumber).call()[5]:
        print("Need decision")
    else:
        print("Don't need decision")
    print("current money:", game.functions.getPlayerByIndex(playerNumber).call()[2])
    print("in prison:",isInPrison(game, playerNumber))
else:
    print("not your move")
    
printLogInfo(filterActionHappened)

players=getPlayers(game)
for player in players:
    print(player,players[player])
getPositions(game)

status 1
Dice value: 5
Need decision
current money: 50
in prison: False
Action: Phill at position 2 : Toll station, pay 50$ to Vlados
Vlados {'money': 50, 'position': 7, 'moves in prison': 0}
Phill {'money': 150, 'position': 2, 'moves in prison': 0}


[['None', ''],
 ['Available', ''],
 ['Bought', 'Vlados'],
 ['None', ''],
 ['None', ''],
 ['Available', ''],
 ['None', ''],
 ['Available', ''],
 ['Available', ''],
 ['None', ''],
 ['None', ''],
 ['Available', '']]

### Make decision about buying station if it is possible

In [29]:
decision = True

In [44]:
print(game.functions.buyStation(decision).call())
tx_hash=game.functions.buyStation(decision).transact()
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
print("status",tx_receipt.status)
events=filterStationBought.get_new_entries()
if len(events) > 0:
    if len(events) == 1:
        print("station",events[len(events)-1]['args']['station'], "is bought by you")
    else:
        print("there are also more events")
else:
    print("station is not bought by you")
    print("current money:", game.functions.getPlayerByIndex(playerNumber).call()[2])
    
printLogInfo(filterActionHappened)

players=getPlayers(game)
for player in players:
    print(player,players[player])
getPositions(game)

True
status 1
station 2 is bought by you
Action: Vlados at position 2 : Station is bought
Vlados {'money': 0, 'position': 2, 'moves in prison': 0}
Phill {'money': 200, 'position': 0, 'moves in prison': 0}


[['None', ''],
 ['Available', ''],
 ['Bought', 'Vlados'],
 ['None', ''],
 ['None', ''],
 ['Available', ''],
 ['None', ''],
 ['Available', ''],
 ['Available', ''],
 ['None', ''],
 ['None', ''],
 ['Available', '']]